# Import All Needed Libraries

In [1]:
!pip install BeautifulSoup4
!pip install requests

from bs4 import BeautifulSoup
import requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

You are using pip version 19.2.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.2.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



#  Step 1: Extract Wikipedia table data

## Extracting raw table from wikipedia webpage

In [2]:
#using beautiful soup to parse the html page
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')
#using soup object, find the html id of table
data = []
columns = []
table = soup.find('table')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    #save the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_data = pd.DataFrame(data = data,columns = columns)
canada_data.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## Data Cleaning

In [3]:
#Ignore cells with a borough that is Not assigned.
canada_data = canada_data[canada_data['Borough'] != 'Not assigned']
canada_data.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
#Combine neighborhoods with the same postal code
canada_data["Neighborhood"] = canada_data.groupby("Postal code")["Neighborhood"].transform(lambda neighbor: ', '.join(neighbor))

In [5]:
#Remove duplicates
canada_data = canada_data.drop_duplicates()

In [6]:
#Update index to be postal code
canada_data = canada_data.set_index('Postal code') 
canada_data.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
canada_data['Neighborhood'].replace("Not assigned", canada_data["Borough"],inplace=True)
canada_data.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
#Print the number of rows of your dataframe.
canada_data.shape[0]

103

#  Step 2: Geocoder

In [9]:
#Get geo data data from csv
geo_data = pd.read_csv("Geospatial_Coordinates.csv")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#Get geo data data from csv
geo_data = pd.read_csv("Geospatial_Coordinates.csv")

#rename columns
geo_data.columns = ["Postal code", "Latitude", "Longitude"]
#set the index to be Postal code
geo_data = geo_data.set_index('Postal code')
#Join the geo data with the toronto main dataframe to get the location per each given postal code
canada_data = canada_data.join(geo_data)
canada_data.head()

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


#  Step 3: Explore and Cluster

## Work with only boroughs that contain the word Toronto

In [11]:
#Filter Canada data to only use boroughs in Toronto
toronto_df = canada_data[canada_data['Borough'].str.contains('Toronto')]
toronto_df.head()

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
M4E,East Toronto,The Beaches,43.676357,-79.293031


## Generate Map to visualize the neighborhoods

In [12]:
toronto_map = folium.Map(location=[toronto_df["Latitude"][0], toronto_df["Longitude"][0]], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(toronto_map)   
toronto_map

## Use FourSquare API to Explore the data

In [13]:
CLIENT_ID = 'IFQ424CPMQT4IKW42R2ZXSMVZSB1YR0CGJI5XPNUDFDUKZ1E' # your Foursquare ID
CLIENT_SECRET = '3XGX0ENCMAKP352PZN2STEIFSLDRYHKOOYAV1TWPBDNDPXBF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IFQ424CPMQT4IKW42R2ZXSMVZSB1YR0CGJI5XPNUDFDUKZ1E
CLIENT_SECRET:3XGX0ENCMAKP352PZN2STEIFSLDRYHKOOYAV1TWPBDNDPXBF


In [14]:
#Create a general function to get the venue data
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):      
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, venue['venue']['name'], venue['venue']['location']['lat'], venue['venue']['location']['lng'], venue['venue']['categories'][0]['name']) for venue in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']  
    return(nearby_venues)

In [15]:
#Get Venue Data from Foursquare for all neighborhoods
LIMIT = 100
radius = 500
toronto_venues_df = getNearbyVenues(names=toronto_df['Neighborhood'],latitudes=toronto_df['Latitude'],longitudes=toronto_df['Longitude'])

In [16]:
toronto_venues_df.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [17]:
print('{} uniques categories.'.format(len(toronto_venues_df['Venue Category'].unique())))

227 uniques categories.


## Analyze Neigborhoods

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_df['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.0000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.017857,0.035714,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.053571,0.000000,0.00000,0.00000,0.000000,0.00,0.00,0.017857,0.000000,0.017857,0.000000,0.017857,0.000,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.017857,0.000000,0.00,0.000000,0.000000,0.035714,0.000000,0.000

In [22]:
#sort venues
def return_top_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    columns.append('#{} Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_top_venues = pd.DataFrame(columns=columns)
neighborhoods_top_venues['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_top_venues.iloc[ind, 1:] = return_top_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_top_venues.head()

,Neighborhood,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Italian Restaurant,Restaurant,Cheese Shop,Seafood Restaurant,Beer Bar,Cocktail Bar,Bakery,Café
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym,Restaurant
2,Business reply mail Processing CentrE,Yoga Studio,Spa,Garden Center,Garden,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Bar,Plane,Boutique,Airport,Airport Food Court,Airport Gate,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Burger Joint,Spa,Ice Cream Shop,Gym / Fitness Center,Japanese Restaurant


## Cluster Neighborhoods

In [23]:
# set number of clusters
kclusters = int(len(toronto_df["Neighborhood"].unique()) / 4)
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)
# add clustering labels
neighborhoods_top_venues.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_df.copy()
#add latitude/longitude for each neighborhood
toronto_data = toronto_merged.join(neighborhoods_top_venues.set_index('Neighborhood'), on='Neighborhood')

toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
Postal code,,,,,,,,,,,,,,,
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Mexican Restaurant,Café,Theater,Spa,Ice Cream Shop
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar,Boutique,Sandwich Place,Burger Joint,Burrito Place,Café
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Italian Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Restaurant,Diner
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Beer Bar,Restaurant,Hotel,American Restaurant,Italian Restaurant,Park,Cosmetics Shop
M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Trail,Health Food Store,Pub,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [30]:
# create map
map_clusters = folium.Map(location=[toronto_df["Latitude"][0], toronto_df["Longitude"][0]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbor, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(str(neighbor) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters (examen & determine the discriminating venue categories that distinguish each cluster)

In [32]:
# Cluster 1
toronto_data.loc[toronto_data['Cluster Labels'] == 0, toronto_data.columns[[1] + list(range(5, toronto_data.shape[1]))]]

,Neighborhood,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
Postal code,,,,,,,,,,,
M5A,Regent Park / Harbourfront,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Mexican Restaurant,Café,Theater,Spa,Ice Cream Shop
M7A,Queen's Park / Ontario Provincial Government,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar,Boutique,Sandwich Place,Burger Joint,Burrito Place,Café
M5B,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Italian Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Restaurant,Diner
M5C,St. James Town,Coffee Shop,Café,Cocktail Bar,Beer Bar,Restaurant,Hotel,American Restaurant,Italian Restaurant,Park,Cosmetics Shop
M5E,Berczy Park,Coffee Shop,Farmers Market,Italian Restaurant,Restaurant,Cheese Shop,Seafood Restaurant,Beer Bar,Cocktail Bar,Bakery,Café
M5G,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Burger Joint,Spa,Ice Cream Shop,Gym / Fitness Center,Japanese Restaurant
M5H,Richmond / Adelaide / King,Coffee Shop,Café,Restaurant,Gym,Hotel,Thai Restaurant,Deli / Bodega,Lounge,Cosmetics Shop,Concert Hall
M6H,Dufferin / Dovercourt Village,Pharmacy,Bakery,Park,Café,Bank,Bar,Middle Eastern Restaurant,Gym / Fitness Center,Supermarket,Grocery Store
M5J,Harbourfront East / Union Station / Toronto Is...,Coffee Shop,Aquarium,Italian Restaurant,Café,Hotel,Restaurant,Sporting Goods Shop,Brewery,Scenic Lookout,Fried Chicken Joint


In [33]:
# Cluster 2
toronto_data.loc[toronto_data['Cluster Labels'] == 1, toronto_data.columns[[1] + list(range(5, toronto_data.shape[1]))]]

,Neighborhood,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
Postal code,,,,,,,,,,,
M4T,Moore Park / Summerhill East,Playground,Tennis Court,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [34]:
# Cluster 3
toronto_data.loc[toronto_data['Cluster Labels'] == 2, toronto_data.columns[[1] + list(range(5, toronto_data.shape[1]))]]

,Neighborhood,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
Postal code,,,,,,,,,,,
M5N,Roselawn,Garden,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [35]:
# Cluster 4
toronto_data.loc[toronto_data['Cluster Labels'] == 3, toronto_data.columns[[1] + list(range(5, toronto_data.shape[1]))]]

,Neighborhood,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
Postal code,,,,,,,,,,,
M4N,Lawrence Park,Bus Line,Swim School,Park,College Gym,College Cafeteria,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
